In [5]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize

np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

year = [2016]
period = ['may-jun'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')
        N = X['N']
        
    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')
        N = X['N']
        
    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        N = X['N']
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')
        N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)


    # Function to normalize the data
    def norm(x):
        
        return (x - np.min(x)) / (np.max(x) - np.min(x))
    
    # Normalize the data
    dist = norm(dist)
    
    a_apr = norm(a_apr)
    a_may = norm(a_may)
    a_jun = norm(a_jun)
    a_jul = norm(a_jul)
    
    
    for period in period:
        print(period)
        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        

        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            
            eta = []
                
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
            
                auto_infection1 = delta1 * (y_lag[i] / n_lag[i]) * np.exp(-eta11 * s_lag[i])
                auto_infection2 = delta2 * (y_lag[i] / n_lag[i]) * np.exp(-eta12 * s_lag[i])

                mask = np.arange(N) != i # mask out the current yard i
                
                dispersal_component1 = gamma1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask]))
            
                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)
            
            eta = np.array(eta).reshape(N,1)
            
            return eta


        def costFunction(theta): 
            
            neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))

            return neg_log_likelihood


        def partial(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                
                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
            
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
            
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)



            grad_entries = np.array([d_delta1, d_delta2, d_gamma1, d_gamma2, d_alpha1, d_alpha2, d_eta11, d_eta12, d_eta21, d_eta22])
            
            return grad_entries

        def partial_by_partial(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))

                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
                

            grad_entries = np.array([[d_delta1*d_delta1, d_delta2*d_delta1, d_gamma1*d_delta1, d_gamma2*d_delta1, d_alpha1*d_delta1, d_alpha2*d_delta1, d_eta11*d_delta1, d_eta12*d_delta1, d_eta21*d_delta1, d_eta22*d_delta1],
                                    [d_delta1*d_delta2, d_delta2*d_delta2, d_gamma1*d_delta2, d_gamma2*d_delta2, d_alpha1*d_delta2, d_alpha2*d_delta2, d_eta11*d_delta2, d_eta12*d_delta2, d_eta21*d_delta2, d_eta22*d_delta2],
                                    [d_delta1*d_gamma1, d_delta2*d_gamma1, d_gamma1*d_gamma1, d_gamma2*d_gamma1, d_alpha1*d_gamma1, d_alpha2*d_gamma1, d_eta11*d_gamma1, d_eta12*d_gamma1, d_eta21*d_gamma1, d_eta22*d_gamma1],
                                    [d_delta1*d_gamma2, d_delta2*d_gamma2, d_gamma1*d_gamma2, d_gamma2*d_gamma2, d_alpha1*d_gamma2, d_alpha2*d_gamma2, d_eta11*d_gamma2, d_eta12*d_gamma2, d_eta21*d_gamma2, d_eta22*d_gamma2],
                                    [d_delta1*d_alpha1, d_delta2*d_alpha1, d_gamma1*d_alpha1, d_gamma2*d_alpha1, d_alpha1*d_alpha1, d_alpha2*d_alpha1, d_eta11*d_alpha1, d_eta12*d_alpha1, d_eta21*d_alpha1, d_eta22*d_alpha1],
                                    [d_delta1*d_alpha2, d_delta2*d_alpha2, d_gamma1*d_alpha2, d_gamma2*d_alpha2, d_alpha1*d_alpha2, d_alpha2*d_alpha2, d_eta11*d_alpha2, d_eta12*d_alpha2, d_eta21*d_alpha2, d_eta22*d_alpha2],
                                    [d_delta1*d_eta11, d_delta2*d_eta11, d_gamma1*d_eta11, d_gamma2*d_eta11, d_alpha1*d_eta11, d_alpha2*d_eta11, d_eta11*d_eta11, d_eta12*d_eta11, d_eta21*d_eta11, d_eta22*d_eta11],
                                    [d_delta1*d_eta12, d_delta2*d_eta12, d_gamma1*d_eta12, d_gamma2*d_eta12, d_alpha1*d_eta12, d_alpha2*d_eta12, d_eta11*d_eta12, d_eta12*d_eta12, d_eta21*d_eta12, d_eta22*d_eta12],
                                    [d_delta1*d_eta21, d_delta2*d_eta21, d_gamma1*d_eta21, d_gamma2*d_eta21, d_alpha1*d_eta21, d_alpha2*d_eta21, d_eta11*d_eta21, d_eta12*d_eta21, d_eta21*d_eta21, d_eta22*d_eta21],
                                    [d_delta1*d_eta22, d_delta2*d_eta22, d_gamma1*d_eta22, d_gamma2*d_eta22, d_alpha1*d_eta22, d_alpha2*d_eta22, d_eta11*d_eta22, d_eta12*d_eta22, d_eta21*d_eta22, d_eta22*d_eta22]])
            
            
            
            return grad_entries

        def partial_sq(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            # delta1 second derivatives
            
            d_delta1_d_eta11 = -tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag) * s_lag
            d_delta1_d_eta12 = 0
            d_delta2_d_eta11 = 0
            d_delta2_d_eta12 = -tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag) * s_lag
            d_gamma1_d_eta22 = 0
            d_gamma1_d_alpha2 = 0
            d_gamma2_d_eta21 = 0
            d_gamma2_d_alpha1 = 0
            d_alpha1_d_gamma2 = 0
            d_alpha1_d_eta22 = 0
            d_alpha1_d_alpha2 = 0
            d_alpha2_d_gamma1 = 0
            d_alpha2_d_eta21 = 0
            d_alpha2_d_alpha1 = 0
            d_eta11_d_delta1 = -tI1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_delta2 = 0
            d_eta11_d_eta11 = tI1 * delta1 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_eta12 = 0
            d_eta12_d_delta1 = 0
            d_eta12_d_delta2 = -tI2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta12_d_eta11 = 0
            d_eta12_d_eta12 = tI2 * delta2 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta21_d_gamma2 = 0
            d_eta21_d_eta22 = 0
            d_eta21_d_alpha2 = 0
            d_eta22_d_gamma1 = 0
            d_eta22_d_eta21 = 0
            d_eta22_d_alpha1 = 0
            
            # summations
            
            d_gamma1_d_eta21 = []
            d_gamma1_d_alpha1 = []
            d_gamma2_d_eta22 = []
            d_gamma2_d_alpha2 = []
            d_alpha1_d_gamma1 = []
            d_alpha1_d_eta21 = []
            d_alpha1_d_alpha1 = []
            d_alpha2_d_gamma2 = []
            d_alpha2_d_eta22 = []
            d_alpha2_d_alpha2 = []
            d_eta21_d_gamma1 = []
            d_eta21_d_eta21 = []
            d_eta21_d_alpha1 = []
            d_eta22_d_gamma2 = []
            d_eta22_d_eta22 = []
            d_eta22_d_alpha2 = []
            
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
                
                d_gamma1_d_eta21_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_gamma1_d_alpha1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_gamma2_d_eta22_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_gamma2_d_alpha2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha1_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_alpha2_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha2_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_eta21_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta21_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (s_lag[mask]**2))
                d_eta21_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))
                d_eta22_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_eta22_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (s_lag[mask]**2))
                d_eta22_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))

                d_gamma1_d_eta21.append(d_gamma1_d_eta21_i)
                d_gamma1_d_alpha1.append(d_gamma1_d_alpha1_i)
                d_gamma2_d_eta22.append(d_gamma2_d_eta22_i)
                d_gamma2_d_alpha2.append(d_gamma2_d_alpha2_i)
                d_alpha1_d_gamma1.append(d_alpha1_d_gamma1_i)
                d_alpha1_d_eta21.append(d_alpha1_d_eta21_i)
                d_alpha1_d_alpha1.append(d_alpha1_d_alpha1_i)
                d_alpha2_d_gamma2.append(d_alpha2_d_gamma2_i)
                d_alpha2_d_eta22.append(d_alpha2_d_eta22_i)
                d_alpha2_d_alpha2.append(d_alpha2_d_alpha2_i)
                d_eta21_d_gamma1.append(d_eta21_d_gamma1_i)
                d_eta21_d_eta21.append(d_eta21_d_eta21_i)
                d_eta21_d_alpha1.append(d_eta21_d_alpha1_i)
                d_eta22_d_gamma2.append(d_eta22_d_gamma2_i)
                d_eta22_d_eta22.append(d_eta22_d_eta22_i)
                d_eta22_d_alpha2.append(d_eta22_d_alpha2_i)
                
            d_gamma1_d_eta21 = np.array(d_gamma1_d_eta21).reshape((N, 1))
            d_gamma1_d_alpha1 = np.array(d_gamma1_d_alpha1).reshape((N, 1))
            d_gamma2_d_eta22 = np.array(d_gamma2_d_eta22).reshape((N, 1))
            d_gamma2_d_alpha2 = np.array(d_gamma2_d_alpha2).reshape((N, 1))
            d_alpha1_d_gamma1 = np.array(d_alpha1_d_gamma1).reshape((N, 1))
            d_alpha1_d_eta21 = np.array(d_alpha1_d_eta21).reshape((N, 1))
            d_alpha1_d_alpha1 = np.array(d_alpha1_d_alpha1).reshape((N, 1))
            d_alpha2_d_gamma2 = np.array(d_alpha2_d_gamma2).reshape((N, 1))
            d_alpha2_d_eta22 = np.array(d_alpha2_d_eta22).reshape((N, 1))
            d_alpha2_d_alpha2 = np.array(d_alpha2_d_alpha2).reshape((N, 1))
            d_eta21_d_gamma1 = np.array(d_eta21_d_gamma1).reshape((N, 1))
            d_eta21_d_eta21 = np.array(d_eta21_d_eta21).reshape((N, 1))
            d_eta21_d_alpha1 = np.array(d_eta21_d_alpha1).reshape((N, 1))
            d_eta22_d_gamma2 = np.array(d_eta22_d_gamma2).reshape((N, 1))
            d_eta22_d_eta22 = np.array(d_eta22_d_eta22).reshape((N, 1))
            d_eta22_d_alpha2 = np.array(d_eta22_d_alpha2).reshape((N, 1))
                
                
            
            zero = np.zeros((N, 1))
            
            hess_entries = np.array([[zero, zero, zero, zero, zero, zero, d_eta11_d_delta1, zero, zero, zero],    #d_delta1
                                    [zero, zero, zero, zero, zero, zero, zero, d_eta12_d_delta2, zero, zero],    #d_delta2
                                    [zero, zero, zero, zero, d_alpha1_d_gamma1, zero, zero, zero, d_eta21_d_gamma1, zero],    #d_gamma1
                                    [zero, zero, zero, zero, zero, d_alpha2_d_gamma2, zero, zero, zero, d_eta22_d_gamma2],    #d_gamma2
                                    [zero, zero, d_gamma1_d_alpha1, zero, d_alpha1_d_alpha1, zero, zero, zero, d_eta21_d_alpha1, zero],    #d_alpha1
                                    [zero, zero, zero, d_gamma2_d_alpha2, zero, d_alpha2_d_alpha2, zero, zero, zero, d_eta22_d_alpha2],    #d_alpha2
                                    [d_delta1_d_eta11, zero, zero, zero, zero, zero, d_eta11_d_eta11, zero, zero, zero],    #d_eta11
                                    [zero, d_delta2_d_eta12, zero, zero, zero, zero, zero, d_eta12_d_eta12, zero, zero],    #d_eta12
                                    [zero, zero, d_gamma1_d_eta21, zero, d_alpha1_d_eta21, zero, zero, zero, d_eta21_d_eta21, zero],    #d_eta21
                                    [zero, zero, zero, d_gamma2_d_eta22, zero, d_alpha2_d_eta22, zero, zero, zero, d_eta22_d_eta22]])   #d_eta22
            
            
            return hess_entries



        # Gradient entries
        def gradient(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Gradient 
            gradient = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
            
            return gradient.ravel()


        # Hessian
        def hessian(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Hessian entries
            hessian = - (1 / N) * np.sum((partial_sq(theta) * mu - n * (partial_by_partial(theta)) * (np.exp(-eta(theta)) / (1 + np.exp(-eta(theta)))**2)), axis=2)
            hessian = hessian.reshape((10, 10))
            
            return hessian
        
        


            
        # Initialize fitting parameters
        theta = np.random.uniform(-6, 6, size=(10, 1))
        print('Theta_0: ', theta.ravel())
        # Minimize the cost function and get the optimized parameter values
        #res = minimize(costFunction, theta, method='Nelder-Mead', jac=gradient, hess=hessian)
        #theta = res.x
        
        # Minimize the cost function and get the optimized parameter values
        res = minimize(costFunction, theta, method='BFGS', jac=gradient, hess=hessian, options={'gtol': 1e-8, 'maxiter': 100000, 'disp': True})
        theta = res.x
        
        # Save trained parameters

        if (year == 2014) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2014_num_nobeta.npy', theta)
        
        elif (year == 2014) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2014_num_nobeta.npy', theta)
            
        elif (year == 2015) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2015_num_nobeta.npy', theta)
            
        elif (year == 2015) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2015_num_nobeta.npy', theta)
            
        elif (year == 2016) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2016_num_nobeta.npy', theta)
            
        elif (year == 2016) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2016_num_nobeta.npy', theta)
            
        elif (year == 2017) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2017_num_nobeta.npy', theta)
            
        elif (year == 2017) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2017_num_nobeta.npy', theta)

may-jun
Theta_0:  [-2.1549534   1.29618919 -1.72741546 -3.16764336 -4.37499873  2.73205804
  1.11757498  3.3705171  -4.18438802  5.86740483]
         Current function value: nan
         Iterations: 2836
         Function evaluations: 3813
         Gradient evaluations: 3797


In [49]:
theta_may_jun = np.load('../reports/parameters/theta_may-jun_2017_num_1.npy', allow_pickle=True)
theta_jun_jul = np.load('../reports/parameters/theta_jun-jul_2017_num_1.npy', allow_pickle=True)

#print("Period: ", "may_jun", "Max Gradient: ", np.max(gradient(theta_may_jun)), "Min Eigenvalue: ", np.min(np.linalg.eigvals(hessian(theta_may_jun))))
#print("Period: ", "jun_jul", "Max Gradient: ", np.max(gradient(theta_jun_jul)), "Min Eigenvalue: ", np.min(np.linalg.eigvals(hessian(theta_jun_jul))))

In [41]:
checkpoint = np.load('../reports/parameters/checkpoints/mle_checkpoint_may-jun_2017_1_1.npy', allow_pickle=True)


In [32]:
theta

array([    -3.78228867,    -12.4045106 ,     15.49532903,   1561.87815082,
       -39588.88698815, 780112.25987214, 397045.99818459,     -0.3939167 ,
           -0.62357441, 642900.74207939, 143279.1319416 ,     37.05528748])

In [29]:
np.linalg.norm(gradient(theta))

7.039354590006846e-09

In [30]:
gradient(theta)

array([-0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.00000001,  0.        , -0.        ,
       -0.        , -0.        ])

In [94]:
theta

array([    2.76562827,     0.38540121, -2026.35615832,    34.42700705,
       -6426.03181702, -9151.57587319,    -4.60772949,    -4.940867  ,
        2184.36919864,    -1.11189886,    -0.4499415 ,     0.17128957])

In [31]:
np.linalg.eigvalsh(hessian(theta))

array([ -0.        ,  -0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.00000039,   0.00166403,
         0.54598605,   1.02481385, 110.7487975 , 223.10232981])

In [7]:
theta_may_jun_2014 = np.load('../reports/parameters/theta_may-jun_2014.npy', allow_pickle=True)
theta_jun_jul_2014 = np.load('../reports/parameters/theta_jun-jul_2014.npy', allow_pickle=True)
theta_may_jun_2015 = np.load('../reports/parameters/theta_may-jun_2015.npy', allow_pickle=True)
theta_jun_jul_2015 = np.load('../reports/parameters/theta_jun-jul_2015.npy', allow_pickle=True)
theta_may_jun_2016 = np.load('../reports/parameters/theta_may-jun_2016.npy', allow_pickle=True)
theta_jun_jul_2016 = np.load('../reports/parameters/theta_jun-jul_2016.npy', allow_pickle=True)
theta_may_jun_2017 = np.load('../reports/parameters/theta_may-jun_2017.npy', allow_pickle=True)
theta_jun_jul_2017 = np.load('../reports/parameters/theta_jun-jul_2017.npy', allow_pickle=True)

In [2]:
theta_may_jun_2017 = np.load('../reports/parameters/theta_may-jun_2017_2.npy', allow_pickle=True)
theta_jun_jul_2017 = np.load('../reports/parameters/theta_jun-jul_2017_2.npy', allow_pickle=True)

In [3]:
theta_may_jun_2017

array([  0.03357575,  -0.08611845, -17.26783211, 442.09613497,
       -10.3583505 ,  -7.69614773,  -0.02299495,  -0.13132382,
        -5.67798122,  -0.12339796,  -0.29866544,   0.52041785])

In [4]:
theta_jun_jul_2017

array([ 0.11050931,  0.14843109, 25.17320391,  8.52103671, -0.00105734,
       -0.44097295, -0.0090511 ,  0.04411858, -0.47797682, -0.16057691,
       -2.27640368, -0.19732548])

In [5]:
gradient(theta_may_jun_2017)

array([[-2.72307692e+00],
       [-1.73076923e+00],
       [-5.69766770e+07],
       [ 5.00551252e-01],
       [-7.38217118e+01],
       [-7.98687122e+01],
       [-1.00987044e+04],
       [-1.30286052e+04],
       [-3.92197000e+09],
       [-2.48510100e+02],
       [-1.61515527e+03],
       [-1.01429263e+03]])

In [ ]:
np.linalg()

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

## Derivatives

$$
\begin{align*}
\frac{\partial \eta_{i}}{\partial \beta_{k}} &= I_{k}^{(t)}(i) \\

\frac{\partial \eta_{i}}{\partial \delta_{k}} &= I_{k}^{(t)}(i)\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{1 k}} &= -I_{k}^{(t)}(i) \delta_{k} s_{i}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}}\right) \exp \left(-\eta_{1 k} s_{i}\right) \\

\frac{\partial \eta_{i}}{\partial \eta_{2 k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z j}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) s_{j}\right] \\

\frac{\partial \eta_{i}}{\partial \gamma_{k}} &= I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j)\right] \\

\frac{\partial \eta_{i}}{\partial \alpha_{k}} &= -\gamma_{k} I_{k}^{(t)}(i) \sum_{j=1}^{M_{i}}\left[\left(\frac{a_{j} z_{j}}{n_{z_{j}}}\right) \exp \left(-\eta_{2 k} s_{j}\right) w_{i j} \exp \left(-\alpha_{k} d_{i j}\right) I_{k}^{(s)}(j) d_{i j}\right]
\end{align*}
$$

### Cost Function

$$J(\theta) = 
-\frac{1}{N} \sum_{i=1}^{N} y_{i} \eta_{i}-n_{i} \log \left(1+e^{\eta_{i}}\right)
$$

### Gradient

$$\frac{\partial J}{\partial \theta} =
-\frac{1}{N}\sum_{i=1}^{N} \frac{\partial \eta_{i}}{\partial \theta}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)
$$


### Hessian

$$\frac{\partial^2 J}{\partial \theta^2} = -\frac{1}{N}
\sum_{i=1}^{N}\left[\frac{\partial^{2} \eta_{i}}{\partial \theta^{2}}\left(y_{i}-\frac{n_{i}}{1+e^{-\eta_{i}}}\right)- n_{i} \left(\frac{\partial \eta_{i}}{\partial \theta}\right)^{2}\frac{e^{-\eta_{i}}}{\left(1+e^{-\eta_{i}}\right)^2}\right]
$$

## Optimization

In [50]:
#checkpoint = np.array([theta, iterations, J_history], dtype=object)
#np.save('../reports/checkpoint_June_July2_backup.npy', checkpoint)

In [45]:
checkpoint = np.load('../reports/checkpoint_June_July2.npy', allow_pickle=True)
theta = checkpoint[0]

In [7]:
theta

array([[-1.52503676],
       [-3.72457905],
       [ 0.65958768],
       [ 9.21605194],
       [-0.25269531],
       [ 0.03692741],
       [ 1.02650034],
       [ 0.84416864],
       [-0.35952126],
       [ 0.26510524],
       [ 1.39258634],
       [ 1.36603416]])

## Parameter Estimation

## Function

$$
\eta_{i}=\sum_{k=1}^{K} I_{k}^{(t)}(i)\left[\beta_{k}+\delta_{k}\left(\frac{\tilde{y}_{i}}{n_{\tilde{y}_{i}}} \exp{\left(-\eta_{1k} s_{i}\right)}\right)+\gamma_{k} \sum_{j=1}^{M_{i}}\left(\frac{a_{j} z_{j}}{n_{z_{j}}} \exp{\left(-\eta_{2k} s_{j}\right)} w_{i j} \exp{\left(-\alpha_{k} d_{i j}\right)} I_{k}^{(s)}(j)\right)\right]
$$

$$\beta_1 = -2.89902093$$
$$\beta_2 = -4.33376942$$
$$\delta_1 = 3.86406603$$
$$\delta_2 = 7.1830044$$
$$\gamma_1 = 0.06209235$$
$$\gamma_2 = 6.21221296$$
$$\alpha_1 = 0.17578305$$
$$\alpha_2 = 1.31264131$$
$$\eta_{11} = 0.13978209$$
$$\eta_{12} = 0.40521989$$
$$\eta_{21} = -0.79182359$$
$$\eta_{22} = 0.55742334$$

## Eigenvalues of Hessian

In [10]:
np.linalg.eigvalsh(hessian(theta))

array([-0.01465556, -0.00072991,  0.00561519,  0.02470412,  0.09781676,
        0.24491857,  0.25933526,  0.32607831,  4.13126881,  5.13835828,
       58.13333007, 84.10888156])

## Gradient

In [145]:
gradient(theta)

array([ 0.        ,  0.00000051, -0.        , -0.        , -0.        ,
       -0.        , -0.        ,  0.        , -0.        , -0.        ])

In [31]:
nGrad = nd.Gradient(costFunction)
nHess = nd.Hessian(costFunction)
hess_theta = nHess(theta.reshape(12,))
grad_theta = nGrad(theta.reshape(12,))

In [10]:
def prob(theta):
    
    p = 1 / (1 + np.exp(-eta(theta)))
    
    return p

In [12]:
print('estimated probability of disease: \n', prob(theta))

estimated probability of disease: 
 [[0.02611759]
 [0.02631787]
 [0.02620498]
 [0.03368977]
 [0.02623533]
 [0.02624147]
 [0.02745905]
 [0.02710209]
 [0.04514933]
 [0.04289552]
 [0.16258389]
 [0.03171659]
 [0.15754236]
 [0.03329544]
 [0.02628519]
 [0.02748307]
 [0.02906667]
 [0.02786479]
 [0.02657169]
 [0.02892042]
 [0.02622809]
 [0.04582204]
 [0.5       ]
 [0.84480418]
 [0.24196183]
 [0.2553849 ]
 [0.33620011]
 [0.02684969]
 [0.49306239]
 [0.80226981]
 [0.53574716]
 [0.41690551]
 [0.91858263]
 [0.14165495]
 [0.        ]
 [0.02439342]
 [0.02535986]
 [0.02457473]
 [0.02567178]
 [0.17610494]
 [0.02407955]
 [0.01639678]
 [0.01495841]
 [0.02234735]
 [0.02345555]
 [0.02276539]
 [0.5       ]
 [0.02272612]
 [0.02457618]
 [0.11173597]
 [0.10794024]
 [0.22234288]
 [0.14008519]
 [0.2470191 ]
 [0.24700966]
 [0.02734554]
 [0.25025533]
 [0.02731197]
 [0.2534015 ]
 [0.02732704]
 [0.24922973]
 [0.02735182]
 [0.02728799]
 [0.02733912]
 [0.02321656]
 [0.02306974]
 [0.02242573]
 [0.5       ]
 [0.02569351

## no beta

In [146]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize

np.set_printoptions(suppress=True)

import warnings
#suppress warnings
warnings.filterwarnings('ignore')

year = [2017]
period = ['jun-jul'] # inputs: 'may-jun', 'jun-jul'

for year in year:
    
    # Import data
    if year == 2014:
        X = np.load('../data/processed/data_2014.npz')
        N = X['N']
        
    elif year == 2015:
        X = np.load('../data/processed/data_2015.npz')
        N = X['N']
        
    elif year == 2016:
        X = np.load('../data/processed/data_2016.npz')
        N = X['N']
        
    elif year == 2017:
        X = np.load('../data/processed/data_2017.npz')
        N = X['N']

    dist = X['distance']
    tI1 = X['tI1'].reshape(N,1)
    tI2 = X['tI2'].reshape(N,1)
    sI2 = X['sI2'].reshape(N,1)
    
    y_apr = X['y_apr'].reshape(N,1)
    y_may = X['y_may'].reshape(N,1)
    y_jun = X['y_jun'].reshape(N,1)
    y_jul = X['y_jul'].reshape(N,1)

    n_apr = X['n_apr'].reshape(N,1)
    n_may = X['n_may'].reshape(N,1)
    n_jun = X['n_jun'].reshape(N,1)
    n_jul = X['n_jul'].reshape(N,1)

    a_apr = X['a_apr'].reshape(N,1)
    a_may = X['a_may'].reshape(N,1)
    a_jun = X['a_jun'].reshape(N,1)
    a_jul = X['a_jul'].reshape(N,1)

    w_apr = X['wind_apr']
    w_may = X['wind_may']
    w_jun = X['wind_jun']
    w_jul = X['wind_jul']

    sI1_apr = X['sI1_apr'].reshape(N,1)
    sI1_may = X['sI1_may'].reshape(N,1)
    sI1_jun = X['sI1_jun'].reshape(N,1)
    sI1_jul = X['sI1_jul'].reshape(N,1)

    s_apr = X['s_apr'].reshape(N,1)
    s_may = X['s_may'].reshape(N,1)
    s_jun = X['s_jun'].reshape(N,1)
    s_jul = X['s_jul'].reshape(N,1)

    n_params = 10
    
    # Function to normalize the data
    def norm(x):
        
        return (x - np.min(x)) / (np.max(x) - np.min(x))
    
    # Normalize the data
    dist = norm(dist)
    
    a_apr = norm(a_apr)
    a_may = norm(a_may)
    a_jun = norm(a_jun)
    a_jul = norm(a_jul)
    
    
    for period in period:

        if period == 'may-jun':
            y = y_jun
            n = n_jun
            y_lag = y_may
            n_lag = n_may
            a_lag = a_may
            w_lag = w_may
            sI1_lag = sI1_may
            s_lag = s_may
        
        elif period == 'jun-jul':
            
            y = y_jul
            n = n_jul
            y_lag = y_jun
            n_lag = n_jun
            a_lag = a_jun
            w_lag = w_jun
            sI1_lag = sI1_jun
            s_lag = s_jun
        

        # Define the function eta() which takes input parameters theta and returns the log-odds of disease for each yard i in current time period
        def eta(theta):
            
            eta = []
                
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            for i in range(0, N):
            
                auto_infection1 = delta1 * (y_lag[i] / n_lag[i]) * np.exp(-eta11 * s_lag[i])
                auto_infection2 = delta2 * (y_lag[i] / n_lag[i]) * np.exp(-eta12 * s_lag[i])

                mask = np.arange(N) != i # mask out the current yard i
                
                dispersal_component1 = gamma1 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask]))
                dispersal_component2 = gamma2 * np.sum(((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask]))
            
                eta_i = tI1[i] * (auto_infection1 + dispersal_component1) + tI2[i] * (auto_infection2 + dispersal_component2)
                eta.append(eta_i)
            
            eta = np.array(eta).reshape(N,1)
            
            return eta


        def costFunction(theta): 
            
            neg_log_likelihood = -(1/N) * np.sum(y * eta(theta) - n * np.log(1 + np.exp(eta(theta))))

            return neg_log_likelihood


        def partial(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                
                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
            
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
            
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)



            grad_entries = np.array([d_delta1, d_delta2, d_gamma1, d_gamma2, d_alpha1, d_alpha2, d_eta11, d_eta12, d_eta21, d_eta22])
            
            return grad_entries

        def partial_by_partial(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            d_beta1 = tI1
            d_beta2 = tI2
            
            d_delta1 = tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_delta2 = tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_eta11 = -tI1 * delta1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta12 = -tI2 * delta2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            
            d_gamma1 = []
            d_gamma2 = []
            d_alpha1 = []
            d_alpha2 = []
            d_eta21 = []
            d_eta22 = []
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
            
                d_gamma1_i = tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask])
                d_gamma2_i = tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask])
                
                d_alpha1_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))

                d_eta21_i = -gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta22_i = -gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                
                d_gamma1.append(d_gamma1_i)
                d_gamma2.append(d_gamma2_i)
                d_alpha1.append(d_alpha1_i)
                d_alpha2.append(d_alpha2_i)
                d_eta21.append(d_eta21_i)
                d_eta22.append(d_eta22_i)
            
            d_gamma1 = np.array(d_gamma1).reshape(N,1)
            d_gamma2 = np.array(d_gamma2).reshape(N,1)
            d_alpha1 = np.array(d_alpha1).reshape(N,1)
            d_alpha2 = np.array(d_alpha2).reshape(N,1)
            d_eta21 = np.array(d_eta21).reshape(N,1)
            d_eta22 = np.array(d_eta22).reshape(N,1)
                

            grad_entries = np.array([[d_beta1*d_beta1, d_beta2*d_beta1, d_delta1*d_beta1, d_delta2*d_beta1, d_gamma1*d_beta1, d_gamma2*d_beta1, d_alpha1*d_beta1, d_alpha2*d_beta1, d_eta11*d_beta1, d_eta12*d_beta1, d_eta21*d_beta1, d_eta22*d_beta1],
                                    [d_beta1*d_beta2, d_beta2*d_beta2, d_delta1*d_beta2, d_delta2*d_beta2, d_gamma1*d_beta2, d_gamma2*d_beta2, d_alpha1*d_beta2, d_alpha2*d_beta2, d_eta11*d_beta2, d_eta12*d_beta2, d_eta21*d_beta2, d_eta22*d_beta2],
                                    [d_beta1*d_delta1, d_beta2*d_delta1, d_delta1*d_delta1, d_delta2*d_delta1, d_gamma1*d_delta1, d_gamma2*d_delta1, d_alpha1*d_delta1, d_alpha2*d_delta1, d_eta11*d_delta1, d_eta12*d_delta1, d_eta21*d_delta1, d_eta22*d_delta1],
                                    [d_beta1*d_delta2, d_beta2*d_delta2, d_delta1*d_delta2, d_delta2*d_delta2, d_gamma1*d_delta2, d_gamma2*d_delta2, d_alpha1*d_delta2, d_alpha2*d_delta2, d_eta11*d_delta2, d_eta12*d_delta2, d_eta21*d_delta2, d_eta22*d_delta2],
                                    [d_beta1*d_gamma1, d_beta2*d_gamma1, d_delta1*d_gamma1, d_delta2*d_gamma1, d_gamma1*d_gamma1, d_gamma2*d_gamma1, d_alpha1*d_gamma1, d_alpha2*d_gamma1, d_eta11*d_gamma1, d_eta12*d_gamma1, d_eta21*d_gamma1, d_eta22*d_gamma1],
                                    [d_beta1*d_gamma2, d_beta2*d_gamma2, d_delta1*d_gamma2, d_delta2*d_gamma2, d_gamma1*d_gamma2, d_gamma2*d_gamma2, d_alpha1*d_gamma2, d_alpha2*d_gamma2, d_eta11*d_gamma2, d_eta12*d_gamma2, d_eta21*d_gamma2, d_eta22*d_gamma2],
                                    [d_beta1*d_alpha1, d_beta2*d_alpha1, d_delta1*d_alpha1, d_delta2*d_alpha1, d_gamma1*d_alpha1, d_gamma2*d_alpha1, d_alpha1*d_alpha1, d_alpha2*d_alpha1, d_eta11*d_alpha1, d_eta12*d_alpha1, d_eta21*d_alpha1, d_eta22*d_alpha1],
                                    [d_beta1*d_alpha2, d_beta2*d_alpha2, d_delta1*d_alpha2, d_delta2*d_alpha2, d_gamma1*d_alpha2, d_gamma2*d_alpha2, d_alpha1*d_alpha2, d_alpha2*d_alpha2, d_eta11*d_alpha2, d_eta12*d_alpha2, d_eta21*d_alpha2, d_eta22*d_alpha2],
                                    [d_beta1*d_eta11, d_beta2*d_eta11, d_delta1*d_eta11, d_delta2*d_eta11, d_gamma1*d_eta11, d_gamma2*d_eta11, d_alpha1*d_eta11, d_alpha2*d_eta11, d_eta11*d_eta11, d_eta12*d_eta11, d_eta21*d_eta11, d_eta22*d_eta11],
                                    [d_beta1*d_eta12, d_beta2*d_eta12, d_delta1*d_eta12, d_delta2*d_eta12, d_gamma1*d_eta12, d_gamma2*d_eta12, d_alpha1*d_eta12, d_alpha2*d_eta12, d_eta11*d_eta12, d_eta12*d_eta12, d_eta21*d_eta12, d_eta22*d_eta12],
                                    [d_beta1*d_eta21, d_beta2*d_eta21, d_delta1*d_eta21, d_delta2*d_eta21, d_gamma1*d_eta21, d_gamma2*d_eta21, d_alpha1*d_eta21, d_alpha2*d_eta21, d_eta11*d_eta21, d_eta12*d_eta21, d_eta21*d_eta21, d_eta22*d_eta21],
                                    [d_beta1*d_eta22, d_beta2*d_eta22, d_delta1*d_eta22, d_delta2*d_eta22, d_gamma1*d_eta22, d_gamma2*d_eta22, d_alpha1*d_eta22, d_alpha2*d_eta22, d_eta11*d_eta22, d_eta12*d_eta22, d_eta21*d_eta22, d_eta22*d_eta22]])
            
            
            
            return grad_entries

        def partial_sq(theta):
            
            delta1, delta2, gamma1, gamma2, alpha1, alpha2, eta11, eta12, eta21, eta22 = theta
            
            # delta1 second derivatives
            
            d_delta1_d_eta11 = -tI1 * (y_lag / n_lag) * np.exp(-eta11 * s_lag) * s_lag
            d_delta1_d_eta12 = 0
            d_delta2_d_eta11 = 0
            d_delta2_d_eta12 = -tI2 * (y_lag / n_lag) * np.exp(-eta12 * s_lag) * s_lag
            d_gamma1_d_eta22 = 0
            d_gamma1_d_alpha2 = 0
            d_gamma2_d_eta21 = 0
            d_gamma2_d_alpha1 = 0
            d_alpha1_d_gamma2 = 0
            d_alpha1_d_eta22 = 0
            d_alpha1_d_alpha2 = 0
            d_alpha2_d_gamma1 = 0
            d_alpha2_d_eta21 = 0
            d_alpha2_d_alpha1 = 0
            d_eta11_d_delta1 = -tI1 * s_lag * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_delta2 = 0
            d_eta11_d_eta11 = tI1 * delta1 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta11 * s_lag)
            d_eta11_d_eta12 = 0
            d_eta12_d_delta1 = 0
            d_eta12_d_delta2 = -tI2 * s_lag * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta12_d_eta11 = 0
            d_eta12_d_eta12 = tI2 * delta2 * (s_lag**2) * (y_lag / n_lag) * np.exp(-eta12 * s_lag)
            d_eta21_d_gamma2 = 0
            d_eta21_d_eta22 = 0
            d_eta21_d_alpha2 = 0
            d_eta22_d_gamma1 = 0
            d_eta22_d_eta21 = 0
            d_eta22_d_alpha1 = 0
            
            # summations
            
            d_gamma1_d_eta21 = []
            d_gamma1_d_alpha1 = []
            d_gamma2_d_eta22 = []
            d_gamma2_d_alpha2 = []
            d_alpha1_d_gamma1 = []
            d_alpha1_d_eta21 = []
            d_alpha1_d_alpha1 = []
            d_alpha2_d_gamma2 = []
            d_alpha2_d_eta22 = []
            d_alpha2_d_alpha2 = []
            d_eta21_d_gamma1 = []
            d_eta21_d_eta21 = []
            d_eta21_d_alpha1 = []
            d_eta22_d_gamma2 = []
            d_eta22_d_eta22 = []
            d_eta22_d_alpha2 = []
            
            
            for i in range(0, N):
                
                mask = np.arange(N) != i # mask out the current yard i
                
                d_gamma1_d_eta21_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_gamma1_d_alpha1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_gamma2_d_eta22_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_gamma2_d_alpha2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha1_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha1_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_alpha2_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1))
                d_alpha2_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * dist[:, i][mask].reshape(N-1, 1) * s_lag[mask])
                d_alpha2_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (dist[:, i][mask].reshape(N-1, 1))**2)
                d_eta21_d_gamma1_i = -tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask])
                d_eta21_d_eta21_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * (s_lag[mask]**2))
                d_eta21_d_alpha1_i = gamma1 * tI1[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta21 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha1 * dist[:, i][mask].reshape(N-1, 1))) * sI1_lag[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))
                d_eta22_d_gamma2_i = -tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask])
                d_eta22_d_eta22_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * (s_lag[mask]**2))
                d_eta22_d_alpha2_i = gamma2 * tI2[i] * np.sum((a_lag[mask] * (y_lag[mask] / n_lag[mask])) * np.exp(-eta22 * s_lag[mask]) * (w_lag[:, i][mask].reshape(N-1, 1) * np.exp(-alpha2 * dist[:, i][mask].reshape(N-1, 1))) * sI2[mask] * s_lag[mask] * (dist[:, i][mask].reshape(N-1, 1)))

                d_gamma1_d_eta21.append(d_gamma1_d_eta21_i)
                d_gamma1_d_alpha1.append(d_gamma1_d_alpha1_i)
                d_gamma2_d_eta22.append(d_gamma2_d_eta22_i)
                d_gamma2_d_alpha2.append(d_gamma2_d_alpha2_i)
                d_alpha1_d_gamma1.append(d_alpha1_d_gamma1_i)
                d_alpha1_d_eta21.append(d_alpha1_d_eta21_i)
                d_alpha1_d_alpha1.append(d_alpha1_d_alpha1_i)
                d_alpha2_d_gamma2.append(d_alpha2_d_gamma2_i)
                d_alpha2_d_eta22.append(d_alpha2_d_eta22_i)
                d_alpha2_d_alpha2.append(d_alpha2_d_alpha2_i)
                d_eta21_d_gamma1.append(d_eta21_d_gamma1_i)
                d_eta21_d_eta21.append(d_eta21_d_eta21_i)
                d_eta21_d_alpha1.append(d_eta21_d_alpha1_i)
                d_eta22_d_gamma2.append(d_eta22_d_gamma2_i)
                d_eta22_d_eta22.append(d_eta22_d_eta22_i)
                d_eta22_d_alpha2.append(d_eta22_d_alpha2_i)
                
            d_gamma1_d_eta21 = np.array(d_gamma1_d_eta21).reshape((N, 1))
            d_gamma1_d_alpha1 = np.array(d_gamma1_d_alpha1).reshape((N, 1))
            d_gamma2_d_eta22 = np.array(d_gamma2_d_eta22).reshape((N, 1))
            d_gamma2_d_alpha2 = np.array(d_gamma2_d_alpha2).reshape((N, 1))
            d_alpha1_d_gamma1 = np.array(d_alpha1_d_gamma1).reshape((N, 1))
            d_alpha1_d_eta21 = np.array(d_alpha1_d_eta21).reshape((N, 1))
            d_alpha1_d_alpha1 = np.array(d_alpha1_d_alpha1).reshape((N, 1))
            d_alpha2_d_gamma2 = np.array(d_alpha2_d_gamma2).reshape((N, 1))
            d_alpha2_d_eta22 = np.array(d_alpha2_d_eta22).reshape((N, 1))
            d_alpha2_d_alpha2 = np.array(d_alpha2_d_alpha2).reshape((N, 1))
            d_eta21_d_gamma1 = np.array(d_eta21_d_gamma1).reshape((N, 1))
            d_eta21_d_eta21 = np.array(d_eta21_d_eta21).reshape((N, 1))
            d_eta21_d_alpha1 = np.array(d_eta21_d_alpha1).reshape((N, 1))
            d_eta22_d_gamma2 = np.array(d_eta22_d_gamma2).reshape((N, 1))
            d_eta22_d_eta22 = np.array(d_eta22_d_eta22).reshape((N, 1))
            d_eta22_d_alpha2 = np.array(d_eta22_d_alpha2).reshape((N, 1))
                
                
            
            zero = np.zeros((N, 1))
            
            hess_entries = np.array([[zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero, zero],    #d_beta2
                                    [zero, zero, zero, zero, zero, zero, zero, zero, d_eta11_d_delta1, zero, zero, zero],    #d_delta1
                                    [zero, zero, zero, zero, zero, zero, zero, zero, zero, d_eta12_d_delta2, zero, zero],    #d_delta2
                                    [zero, zero, zero, zero, zero, zero, d_alpha1_d_gamma1, zero, zero, zero, d_eta21_d_gamma1, zero],    #d_gamma1
                                    [zero, zero, zero, zero, zero, zero, zero, d_alpha2_d_gamma2, zero, zero, zero, d_eta22_d_gamma2],    #d_gamma2
                                    [zero, zero, zero, zero, d_gamma1_d_alpha1, zero, d_alpha1_d_alpha1, zero, zero, zero, d_eta21_d_alpha1, zero],    #d_alpha1
                                    [zero, zero, zero, zero, zero, d_gamma2_d_alpha2, zero, d_alpha2_d_alpha2, zero, zero, zero, d_eta22_d_alpha2],    #d_alpha2
                                    [zero, zero, d_delta1_d_eta11, zero, zero, zero, zero, zero, d_eta11_d_eta11, zero, zero, zero],    #d_eta11
                                    [zero, zero, zero, d_delta2_d_eta12, zero, zero, zero, zero, zero, d_eta12_d_eta12, zero, zero],    #d_eta12
                                    [zero, zero, zero, zero, d_gamma1_d_eta21, zero, d_alpha1_d_eta21, zero, zero, zero, d_eta21_d_eta21, zero],    #d_eta21
                                    [zero, zero, zero, zero, zero, d_gamma2_d_eta22, zero, d_alpha2_d_eta22, zero, zero, zero, d_eta22_d_eta22]])   #d_eta22
            
            
            return hess_entries



        # Gradient entries
        def gradient(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Gradient 
            gradient = - (1 / N) * np.sum((partial(theta) * mu), axis=1)
            
            return gradient.ravel()


        # Hessian
        def hessian(theta):
            
            mu = y - (n / (1 + np.exp(-eta(theta))))
            
            # Hessian entries
            hessian = - (1 / N) * np.sum((partial_sq(theta) * mu - n * (partial_by_partial(theta)) * (np.exp(-eta(theta)) / (1 + np.exp(-eta(theta)))**2)), axis=2)
            hessian = hessian.reshape((12, 12))
            
            return hessian
        
        


            
        # Initialize fitting parameters

        theta = np.random.uniform(-100, 100, size=(10, 1))

        # Minimize the cost function and get the optimized parameter values
        res = minimize(costFunction, theta, method='Nelder-Mead', jac=gradient)
        theta = res.x
        
        # Minimize the cost function and get the optimized parameter values
        res = minimize(costFunction, theta, method='BFGS', jac=gradient, options={'gtol': 1e-6, 'maxiter': 100000, 'disp': True})
        theta = res.x

        # Save trained parameters

        if (year == 2014) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2014_num.npy', theta)
        
        elif (year == 2014) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2014_num.npy', theta)
            
        elif (year == 2015) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2015_num.npy', theta)
            
        elif (year == 2015) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2015_num.npy', theta)
            
        elif (year == 2016) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2016_num.npy', theta)
            
        elif (year == 2016) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2016_num.npy', theta)
            
        elif (year == 2017) & (period == 'may-jun'):
            
            np.save('../reports/parameters/theta_may-jun_2017_num_nobeta.npy', theta)
            
        elif (year == 2017) & (period == 'jun-jul'):
            
            np.save('../reports/parameters/theta_jun-jul_2017_num_nobeta.npy', theta)

Optimization terminated successfully.
         Current function value: 132.919401
         Iterations: 19
         Function evaluations: 34
         Gradient evaluations: 34
